## 使用PaddleX API实现化妆品分类

In [1]:
import paddlex as pdx

import os
# os.environ['CUDA_VISIBLE_DEVICES'] = '0' # 设置使用0号GPU卡（如无GPU，执行此代码后仍然会使用CPU训练模型）
os.environ['CPU_NUM'] = '4'

import matplotlib
matplotlib.use('Agg')

In [2]:
"""定义图像处理流程transforms

定义训练和验证过程中的图像处理流程，其中训练过程包括一些数据增强操作
"""

from paddlex.cls import transforms

train_transforms = transforms.Compose([
    transforms.RandomCrop(crop_size=224),
    transforms.RandomHorizontalFlip(),
    transforms.Normalize()
])

eval_transforms = transforms.Compose([
    transforms.ResizeByShort(short_size=256),
    transforms.CenterCrop(crop_size=224),
    transforms.Normalize()
])

In [3]:
"""定义数据集Dataset

使用PaddleX内置的数据集读取器读取训练和验证数据集，并应用配置的图像处理流程
本示例采用ImageNet数据集格式，因此采用paddlex.datasets.ImageNet来加载数据集
"""

train_dataset = pdx.datasets.ImageNet(
    data_dir='makeup',
    file_list='makeup/train_list.txt',
    label_list='makeup/labels.txt',
    transforms=train_transforms,
    shuffle=True
)

eval_dataset = pdx.datasets.ImageNet(
    data_dir='makeup',
    file_list='makeup/val_list.txt',
    label_list='makeup/labels.txt',
    transforms=eval_transforms
)

2023-01-13 16:27:16 [INFO]	Starting to read file list from dataset...
2023-01-13 16:27:16 [INFO]	710 samples in file makeup/train_list.txt
2023-01-13 16:27:16 [INFO]	Starting to read file list from dataset...
2023-01-13 16:27:16 [INFO]	132 samples in file makeup/val_list.txt


In [4]:
"""训练模型

本示例使用MobileNetV3_large_ssld分类模型
"""

num_classes = len(train_dataset.labels)
model = pdx.cls.MobileNetV3_large_ssld(num_classes=num_classes)

In [5]:
model.train(
    num_epochs=10,
    train_dataset=train_dataset,
    train_batch_size=32,
    eval_dataset=eval_dataset,
    lr_decay_epochs=[4, 6, 8],
    save_interval_epochs=1,
    learning_rate=0.025,
    save_dir='output/mobilenetv3_large_ssld'
)

/opt/anaconda3/lib/python3.9/site-packages/paddle/fluid/layers/math_op_patch.py:336: UserWarning: /opt/anaconda3/lib/python3.9/site-packages/paddlex/cv/nets/mobilenet_v3.py:231
The behavior of expression A * B has been unified with elementwise_mul(X, Y, axis=-1) from Paddle 2.0. If your code works well in the older versions but crashes in this version, try to use elementwise_mul(X, Y, axis=0) instead of A * B. This transitional warning will be dropped in the future.
  warnings.warn(
INFO 2023-01-13 16:27:47,808 optimizer.py:1000] If regularizer of a Parameter has been set by 'fluid.ParamAttr' or 'fluid.WeightNormParamAttr' already. The Regularization[L2Decay, regularization_coeff=0.000100] in Optimizer will not take effect, and it will only be applied to other Parameters!


2023-01-13 16:27:48 [INFO]	Downloading MobileNetV3_large_x1_0_ssld_pretrained.tar from https://paddle-imagenet-models-name.bj.bcebos.com/MobileNetV3_large_x1_0_ssld_pretrained.tar


100%|████████████████████████████████████| 21717/21717 [00:36<00:00, 587.07KB/s]


2023-01-13 16:28:25 [INFO]	Decompressing output/mobilenetv3_large_ssld/pretrain/MobileNetV3_large_x1_0_ssld_pretrained.tar...
2023-01-13 16:28:25 [INFO]	Load pretrain weights from output/mobilenetv3_large_ssld/pretrain/MobileNetV3_large_x1_0_ssld_pretrained.
2023-01-13 16:28:25 [WARNING]	[SKIP] Shape of pretrained weight output/mobilenetv3_large_ssld/pretrain/MobileNetV3_large_x1_0_ssld_pretrained/fc_weights doesn't match.(Pretrained: (1280, 1000), Actual: (1280, 7))
2023-01-13 16:28:25 [WARNING]	[SKIP] Shape of pretrained weight output/mobilenetv3_large_ssld/pretrain/MobileNetV3_large_x1_0_ssld_pretrained/fc_offset doesn't match.(Pretrained: (1000,), Actual: (7,))
2023-01-13 16:28:25 [INFO]	There are 268 varaibles in output/mobilenetv3_large_ssld/pretrain/MobileNetV3_large_x1_0_ssld_pretrained are loaded.


WARNING 2023-01-13 16:28:28,172 reader.py:1441] Your reader has raised an exception!
Exception in thread Thread-17:
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.9/threading.py", line 980, in _bootstrap_inner
    self.run()
  File "/opt/anaconda3/lib/python3.9/threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/anaconda3/lib/python3.9/site-packages/paddle/fluid/reader.py", line 1442, in __thread_main__


SystemError: (Fatal) Blocking queue is killed because the data reader raises an exception.
  [Hint: Expected killed_ != true, but received killed_:1 == true:1.] (at /Users/paddle/xly/workspace/0e927451-9b24-4f4c-91d8-7f5f62245df4/Paddle/paddle/fluid/operators/reader/blocking_queue.h:166)


    six.reraise(*sys.exc_info())
  File "/opt/anaconda3/lib/python3.9/site-packages/six.py", line 719, in reraise
    raise value
  File "/opt/anaconda3/lib/python3.9/site-packages/paddle/fluid/reader.py", line 1422, in __thread_main__
    for tensors in self._tensor_reader():
  File "/opt/anaconda3/lib/python3.9/site-packages/paddle/fluid/reader.py", line 1501, in __tensor_reader_impl__
    for slots in paddle_reader():
  File "/opt/anaconda3/lib/python3.9/site-packages/paddle/fluid/data_feeder.py", line 563, in __reader_creator__
    for item in reader():
  File "/opt/anaconda3/lib/python3.9/site-packages/paddlex/cv/datasets/dataset.py", line 178, in queue_reader
    p.start()
  File "/opt/anaconda3/lib/python3.9/multiprocessing/process.py", line 121, in start
    self._popen = self._Popen(self)
  File "/opt/anaconda3/lib/python3.9/multiprocessing/context.py", line 224, in _Popen
    return _default_context.get_context().Process._Popen(process_obj)
  File "/opt/anaconda3/lib/python3.

In [7]:
"""模型预测"""

result = model.predict('makeup/mascara/27.jpeg', topk=1)

result

[{'category_id': 4, 'category': 'foundation', 'score': 0.30154496}]